In [ ]:
%store -r
import time
import os,json
import requests
from dkube.sdk import *
from dkube.sdk.api import DkubeApi
from dkube.sdk.rsrcs import DkubeModelmonitor
from dkube.sdk.rsrcs.operator import DkubeCluster
from dkube.sdk.rsrcs.modelmonitor import DatasetClass,ModelType,DriftAlgo
from dkube.sdk.rsrcs.modelmonitor import DatasetFormat,DkubeModelmonitoralert, TimeZone
from dkube.sdk.rsrcs.modelmonitor import DataType, ChannelOrder, ImageDataSavedFileFormat

In [ ]:
MONITOR_NAME = image_sgmkr_config['MONITOR_NAME']
DKUBEUSERNAME = image_sgmkr_config['DKUBEUSERNAME']
TOKEN = image_sgmkr_config['TOKEN']
DKUBE_URL = image_sgmkr_config['DKUBE_URL']
RUN_FREQUENCY = image_sgmkr_config['RUN_FREQUENCY']
LIVE_DATASET = image_sgmkr_config['LIVE_DATASET']
TRAINING_DATASET = image_sgmkr_config['TRAINING_DATASET']
ACCESS_KEY = image_sgmkr_config['ACCESS_KEY']
SECRET_KEY = image_sgmkr_config['SECRET_KEY']
BUCKET = image_sgmkr_config['BUCKET']
PREFIX = image_sgmkr_config['PREFIX']
REGION_NAME = image_sgmkr_config['REGION_NAME']
SAGEMAKER_DKUBE_CLUSTER_NAME = image_sgmkr_config['SAGEMAKER_DKUBE_CLUSTER_NAME']

In [ ]:
def get_dataset_version(username, dataset_name, version):
    dataset_versions = api.get_dataset_versions(username, dataset_name)
    versions = []
    for each_version in dataset_versions:
        if each_version["version"]["name"] == version:
            uuid = each_version["version"]["uuid"]
            return f"{version}:{uuid}"
        else:
            versions.append(each_version["version"]["name"])
    return f"dataset version {version} not found, available version are {versions}"

In [ ]:
api = DkubeApi(URL=os.getenv('DKUBE_URL'),token=os.getenv("DKUBE_USER_ACCESS_TOKEN"))

In [ ]:
PREDICT_PREFIX = PREFIX +"/datacapture/"+MONITOR_NAME+"/AllTraffic/"

#### Configure Sagemaker Cluster in Dkube

In [ ]:
pcluster = DkubeCluster(name=SAGEMAKER_DKUBE_CLUSTER_NAME)
pcluster.update_kind("sagemaker")
pcluster.update_class("monitoring")
pcluster.update_authtype("access_keys")
pcluster.update_access_keys(ACCESS_KEY,SECRET_KEY)
pcluster.cluster.access_keys.region = REGION_NAME

In [ ]:
try:
    api.configure_clusters(pcluster.cluster)
except Exception as e:
    print(e)

## Import Deployemnt

In [ ]:
DEPLOYMENT_ID = api.import_deployment(name=MONITOR_NAME,
                                    cluster=SAGEMAKER_DKUBE_CLUSTER_NAME,
                                    variant="AllTraffic")

## Initializing model monitor

In [ ]:
mm=DkubeModelmonitor(deployemnt_id = DEPLOYMENT_ID)

In [ ]:
mm.update_modelmonitor_basics(model_type=ModelType.Classification.value, 
                               input_data_type=DataType.Image.value,
                               data_timezone=TimeZone.UTC.value)

In [ ]:
mm.update_image_data_shape(height=200, width=200, channel=1)

In [ ]:
training_data = f'{TRAINING_DATASET}:'+ DKUBEUSERNAME
train_data_version = get_dataset_version(DKUBEUSERNAME,
                                            TRAINING_DATASET, "v1")
prediction_data = LIVE_DATASET + ":"+ DKUBEUSERNAME
labelled_data = LIVE_DATASET + ":"+ DKUBEUSERNAME
predict_data_format = DatasetFormat.Sagemakerlogs.value

## Adding train, predict, and labelled datasets

In [ ]:
mm.add_datasources(data_class=DatasetClass.Train.value,name=training_data,
                   version=train_data_version)

mm.add_datasources(data_class=DatasetClass.Predict.value,name=prediction_data,
                   s3_subpath=PREDICT_PREFIX, data_format=predict_data_format, date_suffix="yyyy/mm/dd/hh")

mm.add_datasources(data_class=DatasetClass.Labelled.value,name=labelled_data,
                   data_format=DataType.Tabular.value, s3_subpath=MONITOR_NAME+'/groundtruth',
                   predict_col="output",groundtruth_col="label",timestamp_col="timestamp")

## Health Monitoring

In [ ]:
mm.update_deployment_monitoring_details(enabled=True, frequency=1)

## Drift monitoring

In [ ]:
mm.update_drift_monitoring_details(enabled=True,
                                  frequency=RUN_FREQUENCY,
                                  image_train_data_savedfile_format="images_in_labelled_folder")

## Performance Monitoring

In [ ]:
mm.update_performance_monitoring_details(enabled=True,
                                        source_type="labelled_data",
                                        frequency=RUN_FREQUENCY)

## Creating model monitor

In [ ]:
api.modelmonitor_create(mm,wait_for_completion=True)

### Start the model monitor

In [ ]:
api.modelmonitor_start(DEPLOYMENT_ID)

### Cleanup

In [ ]:
CLEANUP = False
if CLEANUP:
    from time import sleep
    RETRIES = 4
    while RETRIES:
        mm = api.modelmonitor_get(DEPLOYMENT_ID)
        if mm["status"] and mm["status"]["state"].lower() != "active":
            break
        elif mm["status"] and mm["status"]["state"].lower() == "active":
            api.modelmonitor_stop(DEPLOYMENT_ID)
        RETRIES -= 1
        sleep(5)
    else:
        raise TimeoutError("modelmonitor failed to stopped")
    api.modelmonitor_delete(DEPLOYMENT_ID)